In [8]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka import KafkaClient, SimpleProducer
from kafka.admin import KafkaAdminClient, NewTopic

import sys

from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener

import requests, json

import tweepy 
import json

In [9]:
ACCESS_TOKEN = '1190219954525736961-HY0tALnzDEHWm2KjrIdCfXi7a5sPJB'
ACCESS_TOKEN_SECRET = 'ZzBPFHBfjGTCgWVP7GvbSucofjCKFbcS4zelu6jiAzt9u'
CONSUMER_KEY = 'nVMEVdzRwWvsoT3uqzHzLvpGq'
CONSUMER_SECRET = 'ZzwSbzC2yeo5daphnSRxQWe2f2Ytamf8boUX4jJ6nQ6gjF1K8n'

In [10]:
kafka = KafkaClient("34.90.212.110:9092")
producer = SimpleProducer(kafka)

In [11]:
location = [-171.791110603, 18.91619, -66.96466, 71.3577635769]

In [20]:
class StdOutListenerWeather(StreamListener):
    
    def on_data(self, status):
        
        tweets = json.loads(status)
        
        api_key = "d475fd06a1edf8578676f78ac590607b"
        base_url = "http://api.openweathermap.org/data/2.5/weather?"
        city_name = "New York"
        complete_url = base_url + "appid=" + api_key + "&q=" + city_name
        response = requests.get(complete_url)
        weather = response.json()
        
        if tweets['geo'] is not None:
            data = {
                    'timestamp': tweets['created_at'], 
                    'id': tweets['id'], 
                    'tweet': tweets['text'],
                    'city': city_name,
                    'temperature': weather['main']['temp']- 272.15,
                    'description': weather['weather'][0]['description']
            }
        
            producer.send_messages("weather", json.dumps(data).encode('utf-8'))
        
        return True
    
    def on_error(self, status):
        print(status)
        
listener = StdOutListenerWeather()
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
stream = Stream(auth, listener)

In [21]:
try:
    print('Start streaming.')
    stream.filter(locations=location, track=['NY', 'New York'],languages=['en'])
except KeyboardInterrupt:
    print("Stopped.")
finally:
    print('Done.')
    stream.disconnect()

Start streaming.
Stopped.
Done.
